# Caracterización de los Clientes Libres en Distribución

La presente caracterización se hace a través de *'df_clientes'*, el cual está hecho a partir de la información proporcionada por El Coordinador a través de su base de datos pública. Los datos del consumo facturado corresponden al promedio mensual entre Junio 2024 y Junio 2025 según lo exigido en el Artículo 1-17 de la NTCSD 2024.

La razón de uso de este periodo (Junio 2024 - Junio 2025) se debe a que es la única base de datos con información sobre la comuna del punto de consumo, aunque no esté completa, faltando 1.125.223 clientes sin información sobre la comuna, sin mencionar que los datos en general están bastante corruptos.

Dada esta incompletitud tan grande, **no será** usada para la determinación final de la demanda por Comuna-Empresa, sino que más bien será de uso referencial y para completitud de los datos de *'df_clientes_regulados'*.

Esta completitud considera; la caracterización por par Comuna-Empresa de los clientes libres que *'df_clientes_regulados'* no considera, y la cantidad de alimentadores por par Comuna-Empresa (para saber la cantidad de UM requeridas para función de monitoreo).

Fuente de los datos: [Página web del Coordinador](https://www.coordinador.cl/mercados/documentos/transferencias-economicas-de-empresas-distribuidoras/catastro-clientes-usuarios-en-distribucion/catastro-2025/)

In [1]:
import pandas as pd 
import numpy as np
import unidecode as ud
from pprint import pprint

In [2]:
# Usuarios de la red de Distribución
df_clientes = pd.read_parquet("./Datos_Dx_procesados/ClientesDx.parquet")     # Original

# Se elimina el resto de los registros que no contienen información sobre la Comuna
df_clientes_norm = df_clientes.dropna(subset=["Comuna"]).copy()
df_clientes_sin_comuna = df_clientes[~df_clientes["id_usuario"].isin(df_clientes_norm["id_usuario"])]

df_clientes_norm

,Distribuidor,EmpalmeCodigo,ClienteVigencia,ClienteCalidadJuridica,Comuna,ConsumoTipoUsuario,ConsumoTipoConsumo,ConsumoTipoSuministro,ConsumoTipoTarifa,ConsumoTension,ConsumoPotencia,ConsumoFacturado,ConexionAlimentador,ConexionSubestacionPrimariaDistribucion,id_usuario
0,edelmag,101693682,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,44.355,ALIMENTADOR 2 6,PUERTO NATALES,0
1,edelmag,101693682,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,10.645,ALIMENTADOR 2 6,PUERTO NATALES,1
2,edelmag,101694022,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,1.936,ALIMENTADOR 2 4,PUERTO NATALES,2
3,edelmag,101694024,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,4.167,ALIMENTADOR 2 4,PUERTO NATALES,3
4,edelmag,101836414,Activo,Persona Natural,natales,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,92.742,ALIMENTADOR 2 2,PUERTO NATALES,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884012,electrica til til,56229,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,16.0,709.000,RUNGUE,RUNGUE,7951469
7884013,electrica til til,56232,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,3.0,275.000,TILTIL,POLPAICO (ENEL),7951470
7884014,electrica til til,56233,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,32.0,0.000,TILTIL,POLPAICO (ENEL),7951471
7884015,electrica til til,56244,Activo,Persona Natural,tiltil,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,13kV,15.0,17.000,CALEU,CALEU,7951472


In [3]:
# Nulos en columnas de interés
print(" -> Nulos originales en la columna \"Comuna\" en df_clientes = {:,}".format(df_clientes["Comuna"].isna().sum()))

 -> Nulos originales en la columna "Comuna" en df_clientes = 1,125,223


In [4]:
# Distribudoras con nulos en la columna Comuna
# Nulos en total en la columna Comuna = 1,125,223
df_con_nulos = df_clientes_sin_comuna[df_clientes_sin_comuna["Comuna"].isna()].copy()
df_con_nulos["Distribuidor"].value_counts()

Distribuidor
saesa                553753
frontel              442093
edelaysen             55932
enel distribucion     39633
luz osorno            32805
coopersol              1007
Name: count, dtype: int64

### Sobre la baja calidad de los datos del tipo de tarifa en *'df_clientes'*

Se demuestra con dos ejemplos (**hay hartos más**) que la columna con información sobre el tipo de tarifa de los clientes en comunas donde opera "Enel" no tiene sentido. Los datos enviados por esta empresa al CEN quizás están corrompidos o sufrieron algún tipo de modificación, por lo tanto, al momento de hacer la limpieza, no se considerará para estos casos el fitro en el tipo de tarifa de los clientes.

In [5]:
# Ejemplo 1 de los registros mal clsificados en la comuna de San Miguel
df_san_miguel = df_clientes_norm[df_clientes_norm["Comuna"] == "san miguel"]
tipo_de_tarifa = df_san_miguel["ConsumoTipoTarifa"].value_counts(dropna=False)

print(" -> Cantidad de resgistros en la comuna de San Miguel = {:,}".format(len(df_san_miguel)))
print(" -> Empresa/s distribuidora/s que lo abastece/n: ", df_san_miguel["Distribuidor"].unique())
print(" -> Proporción de cada tipo de tarifa en San Miguel:")
pprint(tipo_de_tarifa)

 -> Cantidad de resgistros en la comuna de San Miguel = 67,750
 -> Empresa/s distribuidora/s que lo abastece/n:  ['enel distribucion']
 -> Proporción de cada tipo de tarifa en San Miguel:
ConsumoTipoTarifa
AT2PP          58386
Peaje_AT3PP     8138
<NA>             609
BT2PP            487
Peaje_AT2PP       62
BT1b              27
BT4.2             26
BT2PPP            14
BT3PP              1
Name: count, dtype: Int64


In [6]:
# Ejemplo 2 de los registros mal clsificados en la comuna de Cerrillos
df_cerrillos = df_clientes_norm[df_clientes_norm["Comuna"] == "cerrillos"]
tipo_de_tarifa = df_cerrillos["ConsumoTipoTarifa"].value_counts(dropna=False)

print(" -> Cantidad de resgistros en la comuna de Cerrillos = {:,}".format(len(df_cerrillos)))
print(" -> Empresa/s distribuidora/s que lo abastece/n: ", df_san_miguel["Distribuidor"].unique())
print(" -> Proporción de cada tipo de tarifa en Cerrillos:")
pprint(tipo_de_tarifa)

 -> Cantidad de resgistros en la comuna de Cerrillos = 29,325
 -> Empresa/s distribuidora/s que lo abastece/n:  ['enel distribucion']
 -> Proporción de cada tipo de tarifa en Cerrillos:
ConsumoTipoTarifa
AT2PP          28590
<NA>             222
Peaje_AT3PP      160
BT2PP            152
Peaje_AT2PP      136
BT4.2             50
BT1b              12
BT2PPP             3
Name: count, dtype: Int64


## Combinación: '*df_clientes*' x '*df_par_comuna_empresadx*

El *'df_par_comuna_empresadx'* está hecho a partir de la información proporcionada por Comisión Nacional de Energía a través de la NTCSD 2024. Realiza una clasificación sobre la densidad de clientes en el par Comuna-Empresa.

Las clasifica en 5 grupos posibles:

 - "EXTREMADAMENTE BAJA" 
 - "MUY BAJA"
 - "BAJA"
 - "MEDIA"
 - "ALTA"

Fuente de los datos: [Norma Técnica de Calidad de Servicio para Sistemas de Distribución](https://www.cne.cl/wp-content/uploads/2024/05/NTCSDx2024-1.pdf) (Anexo: Clasificación de redes)

In [7]:
# Densidad de clientes en el Par Comuna-Empresa de Distribución
df_par_comuna_empresadx = pd.read_excel("./Datos_Dx_procesados/Par-Comuna-EmpresaDx (procesados).xlsx", index_col=0, engine='openpyxl')
df_par_comuna_empresadx

,Comuna,Distribuidor,Densidad,Mapeo_Densidad
0,aisen,edelaysen,MUY BAJA,2
1,algarrobo,edecsa,EXTREMADAMENTE BAJA,1
2,algarrobo,litoral,BAJA,3
3,alhue,cge,EXTREMADAMENTE BAJA,1
4,alto biobio,frontel,EXTREMADAMENTE BAJA,1
...,...,...,...,...
479,bulnes,cge,EXTREMADAMENTE BAJA,1
480,vilcun,cge,EXTREMADAMENTE BAJA,1
481,putre,coopersol,EXTREMADAMENTE BAJA,1
482,arica,desa,EXTREMADAMENTE BAJA,1


### Diferencias entre ambos dataframes

In [8]:
# Elementos únicos en las columnas ascociadas a Distribuidoras y Comunas 
# df_par_comuna_empresadx
distribuidor_par_unique = set(df_par_comuna_empresadx["Distribuidor"].unique())
comuna_par_unique = set(df_par_comuna_empresadx["Comuna"].unique())

# df_clientes_norm
distribuidor_clientes_unique = set(df_clientes_norm["Distribuidor"].unique())
comuna_clientes_unique = set(df_clientes_norm["Comuna"].unique())

# Comunas y Distribuidoras en los que difieren en ambos dataframes
comunas_distintas = comuna_par_unique.symmetric_difference(comuna_clientes_unique)
empresas_distintas = distribuidor_par_unique.symmetric_difference(distribuidor_clientes_unique)

print(" Cantidad de Comunas evaluadas en df_clientes: ", len(comuna_clientes_unique), 
      "\n Cantidad de Comunas en df_par_comuna_empresadx (información CNE): ", len(comuna_par_unique),
      "\n -> Comunas distintas: ", len(comunas_distintas))

print("\n Cantidad de Distribuidoras evaluadas en df_clientes: ", len(distribuidor_clientes_unique), 
      "\n Cantidad de Distribuidoras en df_par_comuna_empresadx (información CNE): ", len(distribuidor_par_unique),
      "\n -> Empresas distintas: ", len(empresas_distintas))

 Cantidad de Comunas evaluadas en df_clientes:  269 
 Cantidad de Comunas en df_par_comuna_empresadx (información CNE):  330 
 -> Comunas distintas:  61

 Cantidad de Distribuidoras evaluadas en df_clientes:  20 
 Cantidad de Distribuidoras en df_par_comuna_empresadx (información CNE):  28 
 -> Empresas distintas:  8


In [9]:
# Cantidad de pares Comuna-Empresa que contiene cada dataframe
pares_clientes = df_clientes_norm[["Comuna", "Distribuidor"]].drop_duplicates()

print(" -> Cantidad de pares Comuna-Empresa únicos en df_cliente:", len(pares_clientes))
print(" -> Cantidad de pares Comuna-Empresa posibles (info CNE):", len(df_par_comuna_empresadx))

 -> Cantidad de pares Comuna-Empresa únicos en df_cliente: 340
 -> Cantidad de pares Comuna-Empresa posibles (info CNE): 484


In [10]:
# Comunas que están en df_par pero no en df_clientes.
print(" -> Comunas que, en principio, están en df_par_comuna_empresadx pero no en df_clientes:\n")
pprint(sorted([c for c in comunas_distintas if pd.notna(c)]), compact=True, width=120)

# Empresas que están en df_par pero no en df_clientes.
print("\n -> Distribuidoras que, en principio, están en df_par_comuna_empresadx pero no en df_clientes:\n")
pprint(sorted([e for e in empresas_distintas if pd.notna(e)]), compact=True, width=120)

 -> Comunas que, en principio, están en df_par_comuna_empresadx pero no en df_clientes:

['aisen', 'alto biobio', 'ancud', 'angol', 'antuco', 'arauco', 'calbuco', 'canete', 'carahue', 'castro', 'chaiten',
 'chile chico', 'cholchol', 'chonchi', 'cisnes', 'cochamo', 'cochrane', 'coihaique', 'collipulli', 'contulmo', 'corral',
 'curaco de velez', 'curanilahue', 'dalcahue', 'futaleufu', 'hualaihue', 'lago verde', 'lanco', 'lebu', 'lonquimay',
 'los alamos', 'los sauces', 'lota', 'lumaco', 'mariquina', 'melipeuco', 'negrete', 'osorno', 'palena', 'panguipulli',
 'puerto octay', 'puqueldon', 'puren', 'putre', 'puyehue', 'queilen', 'quellon', 'quemchi', 'quilaco', 'quinchao',
 'renaico', 'rio ibanez', 'rio negro', 'saavedra', 'san juan de la costa', 'san rosendo', 'santa juana',
 'teodoro schmidt', 'tirua', 'tolten', 'valdivia']

 -> Distribuidoras que, en principio, están en df_par_comuna_empresadx pero no en df_clientes:

['coopersol', 'desa', 'edelaysen', 'frontel', 'luz andes', 'luz osorno

### Unificación de los datos

In [11]:
# Unificación de los datos df_clientes_norm y df_par_comuna_empresadx (6,758,794 registros en total)
df_clientes_par_merged = pd.merge(df_par_comuna_empresadx, df_clientes_norm, on=["Comuna", "Distribuidor"], how="right")

df_clientes_par_merged

,Comuna,Distribuidor,Densidad,Mapeo_Densidad,EmpalmeCodigo,ClienteVigencia,ClienteCalidadJuridica,ConsumoTipoUsuario,ConsumoTipoConsumo,ConsumoTipoSuministro,ConsumoTipoTarifa,ConsumoTension,ConsumoPotencia,ConsumoFacturado,ConexionAlimentador,ConexionSubestacionPrimariaDistribucion,id_usuario
0,natales,edelmag,BAJA,3.0,101693682,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,44.355,ALIMENTADOR 2 6,PUERTO NATALES,0
1,natales,edelmag,BAJA,3.0,101693682,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,10.645,ALIMENTADOR 2 6,PUERTO NATALES,1
2,natales,edelmag,BAJA,3.0,101694022,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,1.936,ALIMENTADOR 2 4,PUERTO NATALES,2
3,natales,edelmag,BAJA,3.0,101694024,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,4.167,ALIMENTADOR 2 4,PUERTO NATALES,3
4,natales,edelmag,BAJA,3.0,101836414,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,0.22kV,0.0,92.742,ALIMENTADOR 2 2,PUERTO NATALES,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6758789,tiltil,electrica til til,MUY BAJA,2.0,56229,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,16.0,709.000,RUNGUE,RUNGUE,7951469
6758790,tiltil,electrica til til,MUY BAJA,2.0,56232,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,3.0,275.000,TILTIL,POLPAICO (ENEL),7951470
6758791,tiltil,electrica til til,MUY BAJA,2.0,56233,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,23kV,32.0,0.000,TILTIL,POLPAICO (ENEL),7951471
6758792,tiltil,electrica til til,MUY BAJA,2.0,56244,Activo,Persona Natural,Cliente Exclusivo Regulado,Residencial,Cliente Regulado,BT1a,13kV,15.0,17.000,CALEU,CALEU,7951472


## Segmentación de datos de usuarios de la red de distribución

A continuación, se realiza la separación en tres subgrupos de los usuarios en distribución, estos son:

 - Segmento **Residencial**: Compuesto por usuarios de tarifa BT1.
 - Segmento **No Residencial**: Compuesto por usuarios regulados con tarifa distinta a la BT1.
 - Segmento **Libre**: Compuestos por todos los usuarios No regulados pero conectados a la red de distribución.

Esta separación se hará tanto para la totalidad de los usuarios ('*df_clientes*') como para solo aquellos con comuna ('*df_clientes_norm*')

### Cantidad de clientes y energía facturada en distribución (**total**)

Se considerarán estas cantidades (con *'df_clientes'*) netamente para obtener referencialmente la cantidad total de usuarios y energía Residencial, No Residencial y Libre que existen según la información del CEN.

#### Filtros para obtener al cliente residencial

In [12]:
# Tipo de Suministro 
print(df_clientes['ConsumoTipoSuministro'].value_counts())

# Tipo de Usuario
print("\n", df_clientes['ConsumoTipoUsuario'].value_counts())

# Tipo de Consumo
print("\n", df_clientes['ConsumoTipoConsumo'].value_counts())

ConsumoTipoSuministro
Cliente Regulado    4634562
Cliente Libre          9291
Name: count, dtype: Int64

 ConsumoTipoUsuario
Cliente Exclusivo Regulado      7866294
MG                                 8815
Cliente Potencialmente Libre       5067
Cliente Exclusivo Libre            3314
PMGD                                527
Name: count, dtype: Int64

 ConsumoTipoConsumo
Residencial                6256434
Comercial                   272920
Residencial con Negocio     110730
Municipal                    27138
Agricola                     25401
Industrial                   22559
Fiscal                        5367
Hospital                      2036
Otro                          1826
F.F.C.C                        438
Cárcel                         150
Name: count, dtype: Int64


In [13]:
# Filtros para df_clientes (ie, usuarios sin comuna y con comuna)
# Se eliminan los registros que contienen nulos en todas las columnas de interés al mismo tiempo
df_clientes_residenciales = df_clientes.dropna(subset=['ConsumoTipoTarifa', 'ConsumoTipoSuministro', 'ConsumoTipoUsuario', 
                                            'ConsumoTipoConsumo', 'ConsumoFacturado'], how='all')

# Se elimina de la columna Tipo de Tarifa:
# - Usuarios con tarifa no residencial (distintas a BT1a y BT1b) para empresas distintas a "enel distribucion"
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['Distribuidor'] == 'enel distribucion') |
                          (((df_clientes_residenciales['ConsumoTipoTarifa'] == 'BT1a') |
                            (df_clientes_residenciales['ConsumoTipoTarifa'] == 'BT1b')) &
                            (df_clientes_residenciales['Distribuidor'] != 'enel distribucion')) |
                          (df_clientes_residenciales['ConsumoTipoTarifa'].isna())]

# Se eliminan los Clientes Libres de la columna Tipo de Suministro
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['ConsumoTipoSuministro'] == 'Cliente Regulado') |
                            (df_clientes_residenciales['ConsumoTipoSuministro'].isna())]

# Se eliminan los clientes no-exclusivamente regulados de la columna Tipo de Usuario
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['ConsumoTipoUsuario'] == 'Cliente Exclusivo Regulado') |
                            (df_clientes_residenciales['ConsumoTipoUsuario'].isna())]

# Se eliminan los clientes no-residenciales de la columna Tipo de Consumo
df_clientes_residenciales = df_clientes_residenciales[((df_clientes_residenciales['ConsumoTipoConsumo'] == 'Residencial') |
                            (df_clientes_residenciales['ConsumoTipoConsumo'] == 'Residencial con Negocio')) |
                            (df_clientes_residenciales['ConsumoTipoConsumo'].isna())]

# Se elimina de la columna Consumo Facturado:
# - Usuarios generadores (fuera del rango -14400 kWh a 0 kWh, para considerar usuarios con generación distribuida [~20 kW])
# - Usuarios con consumo mayor a 7200 kWh/mes (considerando 720 horas en el mes, y 10 kW como máximo en consumo residencial)
df_clientes_residenciales = df_clientes_residenciales[((-14400 < df_clientes_residenciales['ConsumoFacturado']) & 
                            (df_clientes_residenciales['ConsumoFacturado'] < 7200)) |
                            (df_clientes_residenciales['ConsumoFacturado'].isna())]

# Se elimina de la columna Consumo Potencia:
# - Potencia por sobre los 10 kW (máximo en consumo residencial)
df_clientes_residenciales = df_clientes_residenciales[(df_clientes_residenciales['ConsumoPotencia'] <= 10) |
                            (df_clientes_residenciales['ConsumoPotencia'].isna())]

In [14]:
# Se obtienen los clientes No residenciales a partir de los residenciales
df_clientes_no_residenciales = df_clientes[~df_clientes["id_usuario"].isin(df_clientes_residenciales["id_usuario"])]

# Se corrigen registros mal clasificados como clientes No residenciales
df_clientes_residenciales_no_evidentes = df_clientes_no_residenciales[df_clientes_no_residenciales['ConsumoTipoConsumo'] == 'Residencial']

# Filtro por Consumo Facturado y Potencia
df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[((-14400 < df_clientes_residenciales_no_evidentes['ConsumoFacturado']) & 
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'] < 7200)) |
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'].isna())]

df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[(df_clientes_residenciales_no_evidentes['ConsumoPotencia'] <= 10) |
                                                                                (df_clientes_residenciales_no_evidentes['ConsumoPotencia'].isna())]

df_clientes_residenciales = df_clientes_residenciales.merge(df_clientes_residenciales_no_evidentes, how='outer')
df_clientes_no_residenciales = df_clientes[~df_clientes["id_usuario"].isin(df_clientes_residenciales["id_usuario"])]

# Se filtran los usuarios que no son ni resideinciales ni no residenciales
df_clientes_no_residenciales = df_clientes_no_residenciales[(df_clientes_no_residenciales['ConsumoTipoUsuario'] != 'MG') |
                                                            (df_clientes_no_residenciales['ConsumoTipoUsuario'] != 'PMGD')]

# Considerando que clientes que pueden optar a ser libres deben contar con un consumo superior a 300kW,
# Se toma en cuenta un consumo mensual superior a 216,000 kWh/mes (300 kW * 720 horas)
df_clientes_libres_en_dx = df_clientes_no_residenciales[(df_clientes_no_residenciales['ConsumoFacturado'] > 216_000)]

# Se separan los clientes No residenciales de los Libres en Distribución
df_clientes_no_residenciales = df_clientes_no_residenciales[~df_clientes_no_residenciales["id_usuario"].isin(df_clientes_libres_en_dx["id_usuario"])]

cantidad_clientes_regulados = len(df_clientes_residenciales) + len(df_clientes_no_residenciales)
cantidad_energia_regulados = df_clientes_residenciales['ConsumoFacturado'].sum() + df_clientes_no_residenciales['ConsumoFacturado'].sum()

#### Clientes Regulados en distribución (**totales**)

In [15]:
# Información a junio 2025
# -> Cantidad de clientes.
# -> Cantidad de energía (promedio mensual)
cantidad_clientes_residenciales = len(df_clientes_residenciales)
cantidad_energia_residenciales = df_clientes_residenciales['ConsumoFacturado'].sum()

cantidad_clientes_no_residenciales = len(df_clientes_no_residenciales)
cantidad_energia_no_residenciales = df_clientes_no_residenciales['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
	"\nClientes en distribución:", 
	"\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_regulados), 
	"\n -> Energía facturada: {:,} TWh".format((cantidad_energia_regulados / 1_000_000_000).round(2)),
	"\nClientes residenciales:",
	"\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
	"\n -> Energía facturada: {:,} TWh".format((cantidad_energia_residenciales / 1_000_000_000).round(2)),
	"\nClientes no residenciales:",
	"\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales),
	"\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales / 1_000_000_000).round(2)))

(A fecha de junio 2025) 
Clientes en distribución: 
 -> Cantidad de clientes: 7,883,395 
 -> Energía facturada: 2.54 TWh 
Clientes residenciales: 
 -> Cantidad de clientes: 7,427,582 
 -> Energía facturada: 1.36 TWh 
Clientes no residenciales: 
 -> Cantidad de clientes: 455,813 
 -> Energía facturada: 1.18 TWh


#### Clientes Libres en distribución (**totales**)

In [47]:
cantidad_energia_cliente_libre_dx = df_clientes_libres_en_dx['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
	"\nClientes libres en distribución:",
	"\n -> Cantidad de clientes: {:,}".format(len(df_clientes_libres_en_dx)),
	"\n -> Energía facturada: {:,} TWh".format((cantidad_energia_cliente_libre_dx / 1_000_000_000).round(2)))

(A fecha de junio 2025) 
Clientes libres en distribución: 
 -> Cantidad de clientes: 622 
 -> Energía facturada: 0.72 TWh


### Cantidad de clientes y energía facturada en distribución (**con comuna**)

Se utiliza *'df_clientes_norm'* para separar la información del usuario Residencial, No Residencial y Libre que existen según la información del CEN. Con ello se obtienen las caracterizaciones incompletas por par Comuna-Empresa.

#### Filtros para obtener al cliente residencial

In [17]:
# Filtros para df_clientes_par_merged (ie, usuarios con comuna)
# Se eliminan los registros que contienen nulos en todas las columnas de interés al mismo tiempo
df_clientes_par_merged_residencial = df_clientes_par_merged.dropna(subset=['ConsumoTipoTarifa', 'ConsumoTipoSuministro', 'ConsumoTipoUsuario', 
                                            'ConsumoTipoConsumo', 'ConsumoFacturado'], how='all')

# Se elimina de la columna Tipo de Tarifa:
# - Usuarios con tarifa no residencial (distintas a BT1a y BT1b) para empresas distintas a "enel distribucion"
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['Distribuidor'] == 'enel distribucion') |
                                                (((df_clientes_par_merged_residencial['ConsumoTipoTarifa'] == 'BT1a') |
                                                    (df_clientes_par_merged_residencial['ConsumoTipoTarifa'] == 'BT1b')) &
                                                    (df_clientes_par_merged_residencial['Distribuidor'] != 'enel distribucion')) |
                                                (df_clientes_par_merged_residencial['ConsumoTipoTarifa'].isna())]

# Se eliminan los Clientes Libres de la columna Tipo de Suministro
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['ConsumoTipoSuministro'] == 'Cliente Regulado') |
                            (df_clientes_par_merged_residencial['ConsumoTipoSuministro'].isna())]

# Se eliminan los clientes no-exclusivamente regulados de la columna Tipo de Usuario
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['ConsumoTipoUsuario'] == 'Cliente Exclusivo Regulado') |
                            (df_clientes_par_merged_residencial['ConsumoTipoUsuario'].isna())]

# Se eliminan los clientes no-residenciales de la columna Tipo de Consumo
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[((df_clientes_par_merged_residencial['ConsumoTipoConsumo'] == 'Residencial') |
                            (df_clientes_par_merged_residencial['ConsumoTipoConsumo'] == 'Residencial con Negocio')) |
                            (df_clientes_par_merged_residencial['ConsumoTipoConsumo'].isna())]

# Se elimina de la columna Consumo Facturado:
# - Usuarios generadores (fuera del rango -14400 kWh a 0 kWh, para considerar usuarios con generación distribuida [~20 kW])
# - Usuarios con consumo mayor a 7200 kWh/mes (considerando 720 horas en el mes, y 10 kW como máximo en consumo residencial)
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[((-14400 < df_clientes_par_merged_residencial['ConsumoFacturado']) & 
                            (df_clientes_par_merged_residencial['ConsumoFacturado'] < 7200)) |
                            (df_clientes_par_merged_residencial['ConsumoFacturado'].isna())]

# Se elimina de la columna Consumo Potencia:
# - Potencia por sobre los 10 kW (máximo en consumo residencial)
df_clientes_par_merged_residencial = df_clientes_par_merged_residencial[(df_clientes_par_merged_residencial['ConsumoPotencia'] <= 10) |
                            (df_clientes_par_merged_residencial['ConsumoPotencia'].isna())]

In [18]:
# Se obtienen los clientes No residenciales a partir de los residenciales
df_clientes_par_merged_no_residencial = df_clientes_par_merged[~df_clientes_par_merged["id_usuario"]
                                                               .isin(df_clientes_par_merged_residencial["id_usuario"])]

# Se corrigen registros mal clasificados como clientes No residenciales
df_clientes_residenciales_no_evidentes = df_clientes_par_merged_no_residencial[df_clientes_par_merged_no_residencial['ConsumoTipoConsumo'] == 'Residencial']

# Filtro por Consumo Facturado y Consumo Potencia
df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[((-14400 < df_clientes_residenciales_no_evidentes['ConsumoFacturado']) & 
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'] < 7200)) |
                                                                                 (df_clientes_residenciales_no_evidentes['ConsumoFacturado'].isna())]

df_clientes_residenciales_no_evidentes = df_clientes_residenciales_no_evidentes[(df_clientes_residenciales_no_evidentes['ConsumoPotencia'] <= 10) |
                                                                                (df_clientes_residenciales_no_evidentes['ConsumoPotencia'].isna())]

df_clientes_par_merged_residencial = df_clientes_par_merged_residencial.merge(df_clientes_residenciales_no_evidentes, how='outer')

df_clientes_par_merged_no_residencial = df_clientes_par_merged[~df_clientes_par_merged["id_usuario"].isin(df_clientes_par_merged_residencial["id_usuario"])]

# Se filtran los usuarios que no son ni residenciales ni no residenciales
df_clientes_par_merged_no_residencial = df_clientes_par_merged_no_residencial[(df_clientes_par_merged_no_residencial['ConsumoTipoUsuario'] != 'MG') |
                                                                                 (df_clientes_par_merged_no_residencial['ConsumoTipoUsuario'] != 'PMGD')]

# Considerando que clientes que pueden optar a ser libres deben contar con un consumo superior a 300kW,
# Se toma en cuenta un consumo mensual superior a 216,000 kWh/mes (300 kW * 720 horas)
df_clientes_libres_merged_en_dx = df_clientes_par_merged_no_residencial[(df_clientes_par_merged_no_residencial['ConsumoFacturado'] > 216_000)]

# Se separan los clientes No residenciales de los Libres en Distribución
df_clientes_par_merged_no_residencial = df_clientes_par_merged_no_residencial[~df_clientes_par_merged_no_residencial["id_usuario"]
                                                                              .isin(df_clientes_libres_merged_en_dx["id_usuario"])]

cantidad_clientes_regulados = len(df_clientes_par_merged_residencial) + len(df_clientes_par_merged_no_residencial)
cantidad_energia_regulados = df_clientes_par_merged_residencial['ConsumoFacturado'].sum() + df_clientes_par_merged_no_residencial['ConsumoFacturado'].sum()

#### Clientes Regulados (**con comuna**) en distribución

In [19]:
# Información a junio 2025
# -> Cantidad de clientes.
# -> Cantidad de energía (promedio mensual)
cantidad_clientes_residenciales = len(df_clientes_par_merged_residencial)
cantidad_energia_residenciales = df_clientes_par_merged_residencial['ConsumoFacturado'].sum()

cantidad_clientes_no_residenciales = len(df_clientes_par_merged_no_residencial)
cantidad_energia_no_residenciales = df_clientes_par_merged_no_residencial['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
    "\nClientes regulados:", 
    "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_regulados), 
    "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_regulados / 1_000_000_000).round(2)),
	"\nClientes residenciales:",
	"\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
	"\n -> Energía facturada: {:,} TWh".format((cantidad_energia_residenciales / 1_000_000_000).round(2)),
	"\nClientes no residenciales:",
	"\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales),
	"\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales / 1_000_000_000).round(2)))

(A fecha de junio 2025) 
Clientes regulados: 
 -> Cantidad de clientes: 6,758,368 
 -> Energía facturada: 2.16 TWh 
Clientes residenciales: 
 -> Cantidad de clientes: 6,319,985 
 -> Energía facturada: 1.12 TWh 
Clientes no residenciales: 
 -> Cantidad de clientes: 438,383 
 -> Energía facturada: 1.04 TWh


#### Clientes Libres (**con comuna**) en distribución

In [48]:
cantidad_energia_cliente_libre_dx_con_comuna = df_clientes_libres_merged_en_dx['ConsumoFacturado'].sum()

print("(A fecha de junio 2025)",
	"\nClientes libres en distribución:",
	"\n -> Cantidad de clientes: {:,}".format(len(df_clientes_libres_merged_en_dx)),
	"\n -> Energía facturada: {:,} TWh".format((cantidad_energia_cliente_libre_dx_con_comuna / 1_000_000_000).round(2)))

(A fecha de junio 2025) 
Clientes libres en distribución: 
 -> Cantidad de clientes: 426 
 -> Energía facturada: 0.55 TWh


## Caracterización de los Clientes Libres

En cuanto a la agregación de la demanda de los clientes libres en distribución, considerando que de aquellos de los que se tiene información son 426 de un total de 622 según los datos vistos anteriormente, se ponderará el consumo de los 426 tal que se alcancen los 0,72 TWh totales que representan a los 622 clientes libres según la información del Coordinador (*'df_clientes'*).

In [ ]:
# Se crea el df_caracterizacion_libres con la cantidad de clientes y energía por comuna-empresa
df_caracterizacion_libres = (df_clientes_libres_merged_en_dx.groupby(["Distribuidor", "Comuna"], as_index=False)
                                                          .agg(Cantidad_de_clientes_libreDx=("Comuna", "size"),
                                                               Energia_promedio_mensual_libreDx_kWh=("ConsumoFacturado", 
                                                                                                     lambda x: round(x.sum(), 2))))  

# Se agrega la energía promedio mensual de clientes libres en Dx para complementar la energía de los clientes libres no considerados
# Factor de ajuste de energía
energia_no_considerada = cantidad_energia_cliente_libre_dx - cantidad_energia_cliente_libre_dx_con_comuna
energia_extra_a_agregar_por_cliente_libre = energia_no_considerada / len(df_clientes_libres_merged_en_dx)

print(" -> Energía no considerada (kWh): {:,}".format(energia_no_considerada))
print(" -> Energía extra a agregar por cliente libre (kWh): {:,}".format(energia_extra_a_agregar_por_cliente_libre))

df_caracterizacion_libres["Energia_promedio_mensual_libreDx_kWh"] = (df_caracterizacion_libres["Energia_promedio_mensual_libreDx_kWh"]
                                                                      + energia_extra_a_agregar_por_cliente_libre).round(2)

df_caracterizacion_libres

 -> Energía no considerada (kWh): 169,084,166.9518801
 -> Energía extra a agregar por cliente libre (kWh): 396,911.1900278875


,Distribuidor,Comuna,Cantidad_de_clientes_libreDx,Energia_promedio_mensual_libreDx_kWh
0,cec,curico,10,4203514.19
1,cec,romeral,6,2646471.19
2,cec,teno,16,8199661.19
3,cge,alhue,1,662063.19
4,cge,antofagasta,1,700931.19
...,...,...,...,...
89,luz linares,constitucion,1,990342.19
90,luz linares,linares,5,2789460.19
91,luz linares,yerbas buenas,1,624831.19
92,luz parral,longavi,2,1292741.19


In [66]:
# Se crea el df_cantidad_alimentadores con la cantidad de alimentadores por comuna-empresa
df_cantidad_alimentadores = (df_clientes_norm.groupby(["Distribuidor", "Comuna"], as_index=False)
                                                .agg(Cantidad_de_alimentadores=("ConexionAlimentador", "nunique")))    

# Se juntan las tablas df_caracterizacion_libres y df_cantidad_alimentadores
df_caracterizacion_libres = df_caracterizacion_libres.merge(df_cantidad_alimentadores, on=["Distribuidor", "Comuna"], how="left").fillna(0)

df_caracterizacion_libres

,Distribuidor,Comuna,Cantidad_de_clientes_libreDx,Energia_promedio_mensual_libreDx_kWh,Cantidad_de_alimentadores
0,cec,curico,10,4203514.19,4
1,cec,romeral,6,2646471.19,2
2,cec,teno,16,8199661.19,3
3,cge,alhue,1,662063.19,2
4,cge,antofagasta,1,700931.19,25
...,...,...,...,...,...
89,luz linares,constitucion,1,990342.19,8
90,luz linares,linares,5,2789460.19,10
91,luz linares,yerbas buenas,1,624831.19,10
92,luz parral,longavi,2,1292741.19,6


### Se guardan los datos para su uso posterior

In [67]:
df_caracterizacion_libres.to_excel("Datos_Dx_procesados/Caracterizacion_clientes_libres.xlsx", index=False)